In [16]:
import requests as rq
urlsessions=r'https://f1.tfeed.net/sessions/eng_sessions.js'

x=rq.get(urlsessions)

In [40]:
import pandas as pd
teams=pd.Series(re.findall("'\d\d','\D+','\d{4}_\D+','.+','(.+)'",x.text.replace('\t',''))).unique()
teams_dict={}
for team in teams:
    teams_dict[team]=rq.get('https://f1.tfeed.net/teams/eng_{}.js'.format(team)).text

drivers=pd.Series(re.findall("'\d\d','\D+','\d{4}_\D+','(.+)',.+",x.text.replace('\t',''))).unique()
drivers_dict={}
for driver in drivers:
    drivers_dict[driver]=rq.get('https://f1.tfeed.net/drivers/eng_{}.js'.format(driver)).text

In [44]:
import pickle
dir=r'F:\DataspellProjects\IndonesiaReg\Data\f1-tfeed'
# save dictionary to session_data.pkl file
with open(dir+r'\team_data.pkl', 'wb') as fp:
    pickle.dump(teams_dict, fp)
    print('dictionary saved successfully to file')
with open(dir+r'\drivers_data.pkl', 'wb') as fp:
    pickle.dump(drivers_dict, fp)
    print('dictionary saved successfully to file')

dictionary saved successfully to file
dictionary saved successfully to file


In [30]:
import re
#racesx=re.findall('(\d{4}_\D+)\',',x.text)
races=pd.read_csv(r'..\Data\f1-tfeed\season_data.csv')['session_name']
sessions={}

In [31]:
for num,formats in enumerate(races):

    url=r'https://f1.tfeed.net/sessions/{}/session.js?'.format(formats)
    if formats not in sessions:
        resp=rq.get(url)
        if resp.status_code!=200:
            print('Error at '+formats+' iteration number '+str(num+1))
            continue
        sessions[formats]=resp.text

In [34]:
import pickle
dir=r'F:\DataspellProjects\IndonesiaReg\Data\f1-tfeed'
# save dictionary to session_data.pkl file
with open(dir+'\session_data.pkl', 'wb') as fp:
    pickle.dump(sessions, fp)
    print('dictionary saved successfully to file')

dictionary saved successfully to file


In [35]:
import pickle
dir=r'F:\DataspellProjects\IndonesiaReg\Data\f1-tfeed'
# Read dictionary pkl file
with open(dir+'\session_data.pkl', 'rb') as fp:
    sessions = pickle.load(fp)
    

In [ ]:
import os
import zipfile
from tqdm import tqdm
dir=r'F:\DataspellProjects\IndonesiaReg\Data\f1-tfeed'

errors=[]
for formats in races:
    retries=0
    
    url=r'https://f1.tfeed.net/sessions/{}/session.zip'.format(formats)
    if os.path.exists(dir+'\{}'.format(formats)):
        continue
    print('\nDownloading '+formats)
    os.mkdir(dir+'\{}'.format(formats))
    while retries<3:
        try:
            print('Download attempt number '+str(retries+1))
            chunkSize = 1024
            r = rq.get(url, stream=True)
            with open(dir+'\{}\session.zip'.format(formats), 'wb') as f:
                pbar = tqdm( unit="B", total=int( r.headers['Content-Length'] ) )
                for chunk in r.iter_content(chunk_size=chunkSize): 
                    if chunk: # filter out keep-alive new chunks
                        pbar.update (len(chunk))
                        f.write(chunk)
            with zipfile.ZipFile(dir+'\{}\session.zip'.format(formats), 'r') as zip_ref:
                print('Extracting zip...')
                for file in tqdm(iterable=zip_ref.namelist(), total=len(zip_ref.namelist())):
                    zip_ref.extract(member=file, path=dir+'\{}'.format(formats))
            break
        except:
            retries+=1
            raise e
    if retries==3:
        print('Download {} failed'.format(formats))
        errors.append(formats)
    else:
        print('Download {} success'.format(formats))